In [1]:
import torch
from train import train
from lofi_model import LofiModel
from dataset import MidiDataset
import pretty_midi
import numpy as np
from config import *
print(torch.__version__)
print(torch.version.cuda)

c:\Users\Hyperbook\Desktop\STUDIA\SEM III\Projekt zespolowy\venv\Lib\site-packages\pretty_midi\instrument.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


2.7.1+cu118
11.8


# TRAINING

In [ ]:
if __name__ == "__main__":
    # Device setup
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

    # Dataset setup
    dataset_dir = r"C:\Users\Hyperbook\Desktop\STUDIA\SEM III\Projekt zespolowy\dataset\transformed_dataset"

    # Model setup
    model = LofiModel(device)
    model.to(device)

    # Train
    train(model, 
          dataset_dir, 
          experiment_name="First try multitrack, transformed dataset",
          verbose=True, 
          model_save_path = "./saved_models/multitrack LSTM-VAE (1 min).pth")
    
# 5 epok to 209 minut


Using cuda device


COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


Setting up datasets and dataloaders...
Finished setting up datasets and dataloaders.



COMET INFO: Experiment is live on comet.com https://www.comet.com/ziemmi13/lofi-vae-generator/c1d8dec62a0547d8ba8dc165fd812a4f



Starting training:
The datset has 963 batches
Epoch [1/5]
Training:
	Batch index: 1/963
	Current training Loss: 152532.6562
	Current training Reconstruction Loss: 152527.0781
	Current training KL Loss: 5.5774
	Batch index: 101/963
	Current training Loss: 61430.8252
	Current training Reconstruction Loss: 61427.1075
	Current training KL Loss: 3.7177
	Batch index: 201/963
	Current training Loss: 43867.1617
	Current training Reconstruction Loss: 43865.0763
	Current training KL Loss: 2.0856
	Batch index: 301/963
	Current training Loss: 33158.0681
	Current training Reconstruction Loss: 33156.6384
	Current training KL Loss: 1.4297
	Batch index: 401/963
	Current training Loss: 26972.2283
	Current training Reconstruction Loss: 26971.1334
	Current training KL Loss: 1.0950
	Batch index: 501/963
	Current training Loss: 22948.2113
	Current training Reconstruction Loss: 22947.3246
	Current training KL Loss: 0.8868
	Batch index: 601/963
	Current training Loss: 20165.1235
	Current training Reconstruct

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
model = LofiModel(device)
model.to(device)
model.load_state_dict(torch.load("./saved_models/multitrack LSTM-VAE (1 min).pth"))
model.to(device)
model.eval()

LofiModel(
  (encoder): Encoder(
    (lstm): LSTM(640, 256, num_layers=2, batch_first=True, dropout=0.1)
    (hidden_to_mu): Linear(in_features=512, out_features=256, bias=True)
    (hidden_to_logvar): Linear(in_features=512, out_features=256, bias=True)
  )
  (decoder): Decoder(
    (latent_to_hidden): Linear(in_features=256, out_features=512, bias=True)
    (latent_to_cell): Linear(in_features=256, out_features=512, bias=True)
    (lstm): LSTM(640, 256, num_layers=2, batch_first=True, dropout=0.1)
    (fc_out): Linear(in_features=256, out_features=640, bias=True)
  )
)

## Reconstruction

In [4]:
dataset_dir = r"C:\Users\Hyperbook\Desktop\STUDIA\SEM III\Projekt zespolowy\dataset\transformed_dataset"
dataset = MidiDataset(dataset_dir, verbose=True)

In [5]:
for i in range(5):
    original_sequence_tensor, original_length_val, bpm, filename = dataset[i]
    reconstructions = model.reconstruct(original_sequence_tensor, bpm, save_path=f"reconstructions/{filename}-reconstructed.mid")

Reconstructed multi-instrument MIDI saved to reconstructions/cc57f2da27012c4ee777fe667db60647.mid-reconstructed.mid
Reconstructed multi-instrument MIDI saved to reconstructions/01a400f01c65a687860801daf1320942.mid-reconstructed.mid
Reconstructed multi-instrument MIDI saved to reconstructions/43e39bff9abbb242baf5372d4486bee3.mid-reconstructed.mid
Reconstructed multi-instrument MIDI saved to reconstructions/928fe4af351f39d51735d7f8652ecb31.mid-reconstructed.mid
Reconstructed multi-instrument MIDI saved to reconstructions/428c487a54ea907103f0e80eb3359d89.mid-reconstructed.mid


# CHORD EXTRACTOR DEMO

In [7]:
from music21 import converter

midi_pth = r"C:\Users\Hyperbook\Desktop\STUDIA\SEM III\Projekt zespolowy\dataset\1 2 6 - chord progression.mid"
midi_file = converter.parse(midi_pth)

chords = midi_file.chordify()

chord_progression = []

for c in chords.flat.getElementsByClass("Chord"):
    if not c.isRest:
        root_note = c.root().name
        quality = c.quality
        full_chord_name = c.pitchedCommonName
        offset = c.offset  

        chord_progression.append([root_note, quality, full_chord_name, offset])

chord_progression


FileNotFoundError: Cannot find file in C:\Users\Hyperbook\Desktop\STUDIA\SEM III\Projekt zespolowy\dataset\1 2 6 - chord progression.mid

In [ ]:
import numpy as np
array = np.load("../midi/98f90636c139256f4b7dade28ab87088.npz")
print(array)
for a in array.keys():
    print(a)
    print(array[a])
    print(20*"___")

In [ ]:
import numpy as np
from scipy.sparse import csc_matrix
import matplotlib.pyplot as plt

def load_all_tracks_from_npz(npz_path):
    """
    Load all sparse pianoroll tracks from an LPD .npz file into dense arrays.

    Returns:
        dict[int, np.ndarray]: Mapping from track_id to dense pianoroll (shape: time x pitch)
    """
    data = np.load(npz_path)
    tracks = {}

    for key in data.files:
        if key.endswith("_csc_shape"):
            track_id = int(key.split("_")[1])
            shape = data[f"pianoroll_{track_id}_csc_shape"]
            indptr = data[f"pianoroll_{track_id}_csc_indptr"]
            indices = data[f"pianoroll_{track_id}_csc_indices"]

            if f"pianoroll_{track_id}_csc_data" in data:
                values = data[f"pianoroll_{track_id}_csc_data"]
            else:
                values = np.ones_like(indices, dtype=np.uint8)

            sparse = csc_matrix((values, indices, indptr), shape=shape)
            dense = sparse.toarray()
            tracks[track_id] = dense

    return tracks

def plot_all_tracks(tracks_dict):
    """
    Plot all tracks in the same figure, stacked vertically.
    Each track is shown as a separate pianoroll.
    """
    num_tracks = len(tracks_dict)
    fig, axes = plt.subplots(num_tracks, 1, figsize=(12, 2.5 * num_tracks), sharex=True)

    if num_tracks == 1:
        axes = [axes]

    for i, (track_id, pianoroll) in enumerate(sorted(tracks_dict.items())):
        ax = axes[i]
        ax.imshow(pianoroll.T, aspect='auto', origin='lower', cmap='Greys')
        ax.set_title(f"Track {track_id}")
        ax.set_ylabel("Pitch")
        ax.set_xlabel("Time step")

    plt.tight_layout()
    plt.show()



In [ ]:
npz_path = "../midi/ea04f8ebe8f52d78ab6ce59d9ba04d20.npz"

tracks = load_all_tracks_from_npz(npz_path)
plot_all_tracks(tracks)


In [ ]:
npz_path = "../midi/98f90636c139256f4b7dade28ab87088.npz"

tracks = load_all_tracks_from_npz(npz_path)
plot_all_tracks(tracks)